In [27]:
from scholia import query
import sqlite3
import re
import networkx as nx
from itertools import combinations
test = query.doi_to_qs("10.1371/journal.pone.0147152")

print(test)

['Q30386191']


In [ ]:

db = sqlite3.connect("ARCSLiterature.sqlite3")
cursor = db.cursor()
dbquery = cursor.execute('SELECT DOI, Keywords FROM bibliography;')



for d in dbquery:
    for x in d:
        match = re.findall("citizen science", x, re.IGNORECASE)
        if match:
            print(query.doi_to_qs(d[0]), match)


In [36]:
G = nx.Graph()

dbquery = cursor.execute('SELECT DOI, Keywords FROM bibliography;')
for d in dbquery:
    keywordsperarticle = d[1].split("; ")
    #print("---")
    combos = combinations(keywordsperarticle, 2)
    for c in combos:
        G.add_edge(c[0].lower(), c[1].lower())
        print(c[0].lower(), c[1].lower())
        
nx.write_gexf(G, "keywordcombinations.gexf")

citizen science american pika
citizen science distribution
citizen science glacier national park
citizen science observer variability
citizen science ochotona princeps
citizen science professional
citizen science volunteer
citizen science newtech
american pika distribution
american pika glacier national park
american pika observer variability
american pika ochotona princeps
american pika professional
american pika volunteer
american pika newtech
distribution glacier national park
distribution observer variability
distribution ochotona princeps
distribution professional
distribution volunteer
distribution newtech
glacier national park observer variability
glacier national park ochotona princeps
glacier national park professional
glacier national park volunteer
glacier national park newtech
observer variability ochotona princeps
observer variability professional
observer variability volunteer
observer variability newtech
ochotona princeps professional
ochotona princeps volunteer
ochotona

In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """#defaultView:Graph
SELECT ?node ?nodeLabel ?nodeImage ?childNode ?childNodeLabel ?childNodeImage ?rgb WHERE {
  {
    BIND(wd:Q1093434 AS ?node)
    ?node ?p ?i.
    OPTIONAL { ?node wdt:P18 ?nodeImage. }
    ?childNode ?x ?p.
    ?childNode rdf:type wikibase:Property.
    FILTER(STRSTARTS(STR(?i), "http://www.wikidata.org/entity/Q"))
    FILTER(STRSTARTS(STR(?childNode), "http://www.wikidata.org/entity/P"))
  }
  UNION
  {
    BIND("EFFBD8" AS ?rgb)
    wd:Q1093434 ?p ?childNode.
    OPTIONAL { ?childNode wdt:P18 ?childNodeImage. }
    ?node ?x ?p.
    ?node rdf:type wikibase:Property.
    FILTER(STRSTARTS(STR(?childNode), "http://www.wikidata.org/entity/Q"))
  }
  OPTIONAL {
    ?node wdt:P18 ?nodeImage.
    ?childNode wdt:P18 ?childNodeImage.
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}"""


def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'childNode': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q8379395'}, 'node': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P910'}, 'rgb': {'type': 'literal', 'value': 'EFFBD8'}, 'nodeLabel': {'xml:lang': 'en', 'type': 'literal', 'value': "topic's main category"}, 'childNodeLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Category:Citizen science'}}
{'childNode': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P279'}, 'node': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1093434'}, 'nodeLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'citizen science'}, 'childNodeLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'subclass of'}, 'nodeImage': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Scanning%20the%20cliffs%20near%20Logan%20Pass%20for%20mountain%20goats%20%28Citizen%20Science%29%20%284427399123%29.jpg'}}
{'childNode': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P910'}, 'node': 